In [ ]:
#completed and changed output file and module name
#changed inputs and outputs order
#changed initial checks
#corrected commented state machine
#added _Use_Registered_earlyOutReady_If_Available
#removed sign extension when not needed

In [ ]:
#uses DotProduct_Systolic - v1.0
#uses AdderTree - v1.1x

In [ ]:
import sys
import math

In [ ]:
#N>=1
#N better to be >= HRR, if N<HRR, can decrease HRR (to N)
N = input('Enter N: ')

In [ ]:
#HRR>=2
HRR = input('Enter HRR (empty for 2): ')

In [ ]:
N = int(N)
if(N<1):
    N=1
if not HRR:
    HRR=2
else:
    HRR = int(HRR)
    if(HRR<2):
        HRR=2
NMult = math.ceil(N/HRR); #Number of Multipliers Needed

In [ ]:
#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(NMult))

In [ ]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

In [ ]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')

In [ ]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')

In [ ]:
_Use_Registered_earlyOutReady_If_Available = input('Use Registered earlyOutReady If Available? (empty or 1 to enable, 0 to disable): ')

In [ ]:
if not MAMCS:
    MAMCS = NMult
else:
    MAMCS = int(MAMCS)
if MAMCS>=NMult:
    MAMCS = NMult;
    NumOfMaxSizeComponents = 1
    NonMaxSizeComponentSize = 0
else:
    if(MAMCS<1):
        MAMCS=1
    NumOfMaxSizeComponents = math.floor(NMult/MAMCS)
    if (NMult/MAMCS) == NumOfMaxSizeComponents:
        NonMaxSizeComponentSize = 0
    else:
        NonMaxSizeComponentSize = NMult - MAMCS*NumOfMaxSizeComponents       
NumOfComponents = NumOfMaxSizeComponents + (NonMaxSizeComponentSize!=0)
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1
if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0
if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0
if not _Use_Registered_earlyOutReady_If_Available:
    _Use_Registered_earlyOutReady_If_Available = 1
else:
    _Use_Registered_earlyOutReady_If_Available = int(_Use_Registered_earlyOutReady_If_Available)

In [ ]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./DotProduct_{}_HRx{}_C{}_IR_IL.v".format(N, HRR, MAMCS), 'w+')

In [ ]:
print("`timescale 1ns / 1ps\n", file=of)
print("module DotProduct_{}_HRx{}_C{}_IR_IL".format(N, HRR, MAMCS), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
#print("input clk{}x,".format(HRR), file=of)
print("output reg readyForNewDataSeries = 1,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output reg inSeries = 0,", file=of)
else:
    print("output reg [{}:0] inSeries = 0,".format(lghrr-1), file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output reg outReady = 0,", file=of)
lgn=math.ceil(math.log2(N))
al=-2+lgn
if al==0:
    print("output reg signed [(2*IN_WIDTH):0] DP,", file=of)
elif al<0:
    print("output reg signed [(2*IN_WIDTH){}:0] DP,".format(al), file=of)
else:
    print("output reg signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

In [ ]:
lgMAMCS=math.ceil(math.log2(MAMCS))
palM=-2+lgMAMCS
print("reg LLinReady = 0;", file=of)
print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult-1):
    print("SA{}, ".format(i), end='', file=of)
print("SA{};".format(NMult-1), file=of)
print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult-1):
    print("SB{}, ".format(i), end='', file=of)
print("SB{};".format(NMult-1), file=of)
print(file=of)

In [ ]:
if MAMCS>1:
    print("wire DPOutReady;", file=of)
    for i in range (NumOfMaxSizeComponents):
        if palM==0:
            print("wire signed [(2*IN_WIDTH):0] DPpart", end='', file=of)
        elif palM<0:
            print("wire signed [(2*IN_WIDTH){}:0] DPpart".format(palM), end='', file=of)
        else:
            print("wire signed [(2*IN_WIDTH)+{}:0] DPpart".format(palM), end='', file=of)
        if NumOfComponents>1:
            print("{};".format(i), file=of)
        else:
            print(";", file=of)
        print("DotProduct_Systolic_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(MAMCS), file=of)
        print("\tDPS{}_{}(clk, reset, enable,".format(MAMCS, i), file=of)
        print("\tLLinReady,", file=of)
        print("\t", end='', file=of)
        for j in range(MAMCS):
            print("SA{}, ".format(MAMCS*i+j), end='', file=of)
        print(file=of)
        print("\t", end='', file=of)
        for j in range(MAMCS):
            print("SB{}, ".format(MAMCS*i+j), end='', file=of)
        print(file=of)
        if i==0:
            print("\tDPOutReady,", file=of)
        else:
            print("\tDPOutReady{}, //not used".format(i), file=of)
        print("\tDPpart", end='', file=of)
        if NumOfComponents>1:
            print("{});\n".format(i), file=of)
        else:
            print(");\n", file=of)

In [ ]:
if MAMCS==1:
    print("wire signed [(2*IN_WIDTH)-1:0] ", end='', file=of) #-2 is enough, XST
    for i in range (NumOfMaxSizeComponents):    
        print("DPpart".format(i), end='', file=of)
        if NumOfComponents>1:
            print("{}".format(i), end='', file=of)
        if i==NumOfMaxSizeComponents-1:
            print(";", file=of)              
        else:
            print(", ", end='', file=of)
    print("wire DPOutReady;\n", file=of)

In [ ]:
if MAMCS==1:
    print("generate", file=of)
    print("if(INPUT_REG_DEPTH==0 && MULT_PIPE_DEPTH==0) begin\n", file=of)
    if NumOfComponents>1:
        for i in range (NumOfMaxSizeComponents):    
            print("assign DPpart{} = SA{}*SB{};".format(i, i, i),file=of)
    else:
        print("assign DPpart = SA0*SB0;", file=of)
    print("assign DPOutReady = LLinReady;\n", file=of)
    print("end", file=of)

In [ ]:
if MAMCS==1:
    print("else if(INPUT_REG_DEPTH==0 && MULT_PIPE_DEPTH!=0) begin\n", file=of)
    print("reg signed [(2*IN_WIDTH)-1:0] ", end='', file=of) #-2 is enough, XST
    for i in range (NumOfMaxSizeComponents):    
        print("DPpartR{}[0:MULT_PIPE_DEPTH-1]".format(i), end='', file=of)
        if i==NumOfMaxSizeComponents-1:
            print(";", file=of)              
        else:
            print(", ", end='', file=of)
    print("integer i;", file=of)
    print("reg [0:MULT_PIPE_DEPTH-1] DPOutReadyR = 0;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tDPOutReadyR <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    for i in range (NumOfMaxSizeComponents):   
        print("\t\tDPpartR{}[0]".format(i), end='', file=of)
        print(" <= SA{}*SB{};".format(i, i), file=of)
    print("\t\tDPOutReadyR[0] <= LLinReady;", file=of)
    print("\t\tfor(i=0;i<(MULT_PIPE_DEPTH-1);i=i+1) begin", file=of)
    for i in range (NumOfMaxSizeComponents):
        print("\t\t\tDPpartR{}[i+1] <= DPpartR{}[i];".format(i, i), file=of)
    print("\t\t\tDPOutReadyR[i+1] <= DPOutReadyR[i];", file=of)    
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    if NumOfComponents==1:
        print("assign DPpart = DPpartR0[MULT_PIPE_DEPTH-1];", file=of)
    else:
        for i in range (NumOfMaxSizeComponents):    
            print("assign DPpart{} = DPpartR{}[MULT_PIPE_DEPTH-1];".format(i, i),file=of)
    print("assign DPOutReady = DPOutReadyR[MULT_PIPE_DEPTH-1];\n", file=of)
    print("end", file=of)

In [ ]:
if MAMCS==1:
    print("else if(INPUT_REG_DEPTH!=0 && MULT_PIPE_DEPTH==0) begin\n", file=of)
    for i in range (NumOfMaxSizeComponents):   
        print("reg signed [IN_WIDTH-1:0] SA{}r [1:INPUT_REG_DEPTH], SB{}r [1:INPUT_REG_DEPTH];".format(i, i), file=of)
    print("integer i, j;", file=of)
    print("reg [0:INPUT_REG_DEPTH-1] DPOutReadyR = 0;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tDPOutReadyR <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    for i in range(NumOfMaxSizeComponents):
        print("\t\tSA{}r[1] <= SA{};".format(i,i), file=of)
        print("\t\tSB{}r[1] <= SB{};".format(i,i), file=of)    
    print("\t\tfor(j=1;j<INPUT_REG_DEPTH;j=j+1) begin", file=of)
    for i in range(NumOfMaxSizeComponents):
        print("\t\t\tSA{}r[j+1] <= SA{}r[j];".format(i,i), file=of)
        print("\t\t\tSB{}r[j+1] <= SB{}r[j];".format(i,i), file=of)
    print("\t\tend", file=of)
    print("\t\tDPOutReadyR[0] <= LLinReady;", file=of)
    print("\t\tfor(i=0;i<(INPUT_REG_DEPTH-1);i=i+1) begin", file=of)
    print("\t\t\tDPOutReadyR[i+1] <= DPOutReadyR[i];", file=of)    
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    if NumOfComponents>1:
        for i in range (NumOfMaxSizeComponents):    
            print("assign DPpart{} = SA{}r[INPUT_REG_DEPTH]*SB{}r[INPUT_REG_DEPTH];".format(i, i, i),file=of)
    else:
        print("assign DPpart = SA0r[INPUT_REG_DEPTH]*SB0r[INPUT_REG_DEPTH];", file=of)
    print("assign DPOutReady = DPOutReadyR[INPUT_REG_DEPTH-1];\n", file=of)
    print("end", file=of)

In [ ]:
if MAMCS==1:
    print("else begin //if(INPUT_REG_DEPTH!=0 && MULT_PIPE_DEPTH!=0)\n", file=of)
    for i in range (NumOfMaxSizeComponents):   
        print("reg signed [IN_WIDTH-1:0] SA{}r [1:INPUT_REG_DEPTH], SB{}r [1:INPUT_REG_DEPTH];".format(i, i), file=of)
    print("reg signed [(2*IN_WIDTH)-1:0] ", end='', file=of) #-2 is enough, XST
    for i in range (NumOfMaxSizeComponents):    
        print("DPpartR{}[0:MULT_PIPE_DEPTH-1]".format(i), end='', file=of)
        if i==NumOfMaxSizeComponents-1:
            print(";", file=of)              
        else:
            print(", ", end='', file=of)
    print("integer i, j, k;", file=of)
    print("reg [0:INPUT_REG_DEPTH+MULT_PIPE_DEPTH-1] DPOutReadyR = 0;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tDPOutReadyR <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    for i in range(NumOfMaxSizeComponents):
        print("\t\tSA{}r[1] <= SA{};".format(i,i), file=of)
        print("\t\tSB{}r[1] <= SB{};".format(i,i), file=of)    
    print("\t\tfor(j=1;j<INPUT_REG_DEPTH;j=j+1) begin", file=of)
    for i in range(NumOfMaxSizeComponents):
        print("\t\t\tSA{}r[j+1] <= SA{}r[j];".format(i,i), file=of)
        print("\t\t\tSB{}r[j+1] <= SB{}r[j];".format(i,i), file=of)
    print("\t\tend", file=of)
    for i in range (NumOfMaxSizeComponents):   
        print("\t\tDPpartR{}[0]".format(i), end='', file=of)
        print(" <= SA{}r[INPUT_REG_DEPTH]*SB{}r[INPUT_REG_DEPTH];".format(i, i), file=of)
    print("\t\tfor(k=0;k<(MULT_PIPE_DEPTH-1);k=k+1) begin", file=of)
    for i in range (NumOfMaxSizeComponents):
        print("\t\t\tDPpartR{}[k+1] <= DPpartR{}[k];".format(i, i), file=of)
    print("\t\tend", file=of)
    print("\t\tDPOutReadyR[0] <= inReady;", file=of)
    print("\t\tfor(i=0;i<(INPUT_REG_DEPTH+MULT_PIPE_DEPTH-1);i=i+1) begin", file=of)
    print("\t\t\tDPOutReadyR[i+1] <= DPOutReadyR[i];", file=of)    
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    if NumOfComponents==1:
        print("assign DPpart = DPpartR0[MULT_PIPE_DEPTH-1];", file=of)
    else:
        for i in range (NumOfMaxSizeComponents):    
            print("assign DPpart{} = DPpartR{}[MULT_PIPE_DEPTH-1];".format(i, i),file=of)
    print("assign DPOutReady = DPOutReadyR[INPUT_REG_DEPTH+MULT_PIPE_DEPTH-1];\n", file=of)
    print("end", file=of)
    print("endgenerate\n", file=of)

In [ ]:
if NonMaxSizeComponentSize>1:
    LGNonMaxSizeComponentSize=math.ceil(math.log2(NonMaxSizeComponentSize))
    palN=-2+LGNonMaxSizeComponentSize
    if palN==0:
        print("wire signed [(2*IN_WIDTH):0] DPpart{};".format(NumOfMaxSizeComponents), file=of)
    elif palN<0:
        print("wire signed [(2*IN_WIDTH){}:0] DPpart{};".format(palN, NumOfMaxSizeComponents), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{};".format(palN, NumOfMaxSizeComponents), file=of)
    print("DotProduct_Systolic_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH+{}), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(NonMaxSizeComponentSize, MAMCS-NonMaxSizeComponentSize), file=of)
    print("\tDPS{}(clk, reset, enable,".format(NonMaxSizeComponentSize), file=of)
    print("\tLLinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("SA{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("SB{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\tDPOutReady{}, //not used".format(NumOfMaxSizeComponents), file=of)
    print("\tDPpart{});".format(NumOfMaxSizeComponents), file=of)
    #naming and sign extend (if needed)
    if palM-palN>0:
        if palM==0:
            print("wire signed [(2*IN_WIDTH):0] DPpart{}E = ".format(NumOfMaxSizeComponents), end='', file=of)    
        elif palM<0:
            print("wire signed [(2*IN_WIDTH){}:0] DPpart{}E = ".format(palM, NumOfMaxSizeComponents), end='', file=of)    
        else:
            print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{}E = ".format(palM, NumOfMaxSizeComponents), end='', file=of)
    #if palM-palN==0:
    #    print("DPpart{};".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    #else:
        if palN==0:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH)]}}}},DPpart{}}};".format(palM-palN, NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
        elif palN<0:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH){}]}}}},DPpart{}}};".format(palM-palN, NumOfMaxSizeComponents, palN, NumOfMaxSizeComponents), file=of)                
        else:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH)+{}]}}}},DPpart{}}};".format(palM-palN, NumOfMaxSizeComponents, palN, NumOfMaxSizeComponents), file=of)    
    print(file=of)

In [ ]:
if NonMaxSizeComponentSize==1:
    print("reg signed [IN_WIDTH-1:0] SA{}r[1:{}+INPUT_REG_DEPTH], SB{}r[1:{}+INPUT_REG_DEPTH];".format(MAMCS*NumOfMaxSizeComponents, MAMCS-1, MAMCS*NumOfMaxSizeComponents, MAMCS-1), file=of)
    if palM==0:
        print("wire signed [(2*IN_WIDTH):0] DPpart{}E;".format(NumOfMaxSizeComponents), file=of)    
    elif palM<0:
        print("wire signed [(2*IN_WIDTH){}:0] DPpart{}E;".format(palM, NumOfMaxSizeComponents), file=of)    
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{}E;".format(palM ,NumOfMaxSizeComponents), file=of)
    print("generate", file=of)
    print("if(MULT_PIPE_DEPTH==0) begin\n", file=of)
    print("integer i;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(enable) begin", file=of)
    print("\t\tSA{}r[1] <= SA{};".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tSB{}r[1] <= SB{};".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tfor(i=1;i<({}+INPUT_REG_DEPTH);i=i+1) begin".format(MAMCS-1), file=of)
    print("\t\t\tSA{}r[i+1] <= SA{}r[i];".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\t\tSB{}r[i+1] <= SB{}r[i];".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print("wire signed [(2*IN_WIDTH)-1:0] DPpart{} = SA{}r[{}+INPUT_REG_DEPTH]*SB{}r[{}+INPUT_REG_DEPTH];".format(NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents, MAMCS-1, MAMCS*NumOfMaxSizeComponents, MAMCS-1), file=of)
    #naming and sign extend (if needed)
    if MAMCS==2: #palM==-1
        print("assign DPpart{}E = DPpart{};\n".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    else:
        print("assign DPpart{}E = {{{{{}{{DPpart{}[(2*IN_WIDTH)-1]}}}},DPpart{}}};\n"
        .format(NumOfMaxSizeComponents, palM+1, NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    print("end", file=of)

In [ ]:
if NonMaxSizeComponentSize==1:
    print("else begin //if(MULT_PIPE_DEPTH!=0)\n", file=of)
    print("reg signed [(2*IN_WIDTH)-1:0] DPpart{}[0:MULT_PIPE_DEPTH-1];".format(NumOfMaxSizeComponents), file=of)  #-2 is enough, XST
    print("integer i, j;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(enable) begin", file=of)
    print("\t\tSA{}r[1] <= SA{};".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tSB{}r[1] <= SB{};".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tfor(i=1;i<({}+INPUT_REG_DEPTH);i=i+1) begin".format(MAMCS-1), file=of)
    print("\t\t\tSA{}r[i+1] <= SA{}r[i];".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\t\tSB{}r[i+1] <= SB{}r[i];".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("\t\tend", file=of)    
    print("\t\tDPpart{}[0] <= SA{}r[{}+INPUT_REG_DEPTH]*SB{}r[{}+INPUT_REG_DEPTH];".format(NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents, MAMCS-1, MAMCS*NumOfMaxSizeComponents, MAMCS-1), file=of)
    print("\t\tfor(j=0;j<MULT_PIPE_DEPTH-1;j=j+1) begin", file=of)
    print("\t\t\tDPpart{}[j+1] <= DPpart{}[j];".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    #naming and sign extend (if needed)
    if MAMCS==2: #palM==-1
        print("assign DPpart{}E = DPpart{}[MULT_PIPE_DEPTH-1];\n".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    else:
        print("assign DPpart{}E = {{{{{}{{DPpart{}[MULT_PIPE_DEPTH-1][(2*IN_WIDTH)-1]}}}},DPpart{}[MULT_PIPE_DEPTH-1]}};\n"
        .format(NumOfMaxSizeComponents, palM+1, NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    print("end", file=of)
    print("endgenerate\n", file=of)

In [ ]:
palA=-2+math.ceil(math.log2(NMult))
if NumOfComponents>1:
    print("wire ATOutReady;", file=of)
    palA=-2+math.ceil(math.log2(NMult))
    if palA==0:
        print("wire signed [2*IN_WIDTH:0] DPpart;", file=of)
    elif palA<0:
        print("wire signed [2*IN_WIDTH{}:0] DPpart;".format(palA), file=of)
    else:
        print("wire signed [2*IN_WIDTH+{}:0] DPpart;".format(palA), file=of)
    print("wire ATEarlyOutReady;", file=of)
    if lgMAMCS==1:
        print("AdderTree_{}#( .IN_WIDTH(2*IN_WIDTH) )".format(NumOfComponents), file=of)
    elif lgMAMCS<1:
        print("AdderTree_{}#( .IN_WIDTH(2*IN_WIDTH{}) )".format(NumOfComponents, -1+lgMAMCS), file=of)
    else:
        print("AdderTree_{}#( .IN_WIDTH(2*IN_WIDTH+{}) )".format(NumOfComponents, -1+lgMAMCS), file=of)
    print("\tAT(clk, reset, enable, DPOutReady,\n\t", end='', file=of)
    for i in range(NumOfMaxSizeComponents):
        print("DPpart{}, ".format(i), end='', file=of)
    if NonMaxSizeComponentSize!=0:
        print("DPpart{}".format(NumOfMaxSizeComponents), end='', file=of)
        if palM-palN>0 or NonMaxSizeComponentSize==1:
            print("E", end='', file=of)
        print(",", end='', file=of)
    print("\n\tATOutReady, DPpart,", file=of)
    print("\tATEarlyOutReady);\n", file=of)

In [ ]:
if(N>NMult):
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult,N-1):
        print("A{}l, ".format(i), end='', file=of)
    print("A{}l;".format(N-1), file=of)
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult,N-1):
        print("B{}l, ".format(i), end='', file=of)
    print("B{}l;".format(N-1), file=of)
#NZ = HRR*NMult - N
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\treadyForNewDataSeries <= 1;", file=of)
print("\t\tinSeries <= 0;", file=of)
print("\t\tLLinReady <= 0;", file=of)
print("\tend", file=of)
print("\telse if(enable) begin", file=of)
print("\t\tif(inSeries==0) begin".format(HRR-1), file=of)
print("\t\t\tLLinReady <= inReady;", file=of)
print("\t\t\tif(inReady) begin", file=of)
# print("\t\t\t\tLLinReady <= 1;", file=of)
print("\t\t\t\treadyForNewDataSeries <= 0;", file=of)
if HRR==2:
    print("\t\t\t\tinSeries <= 1;", file=of)    
else:
    print("\t\t\t\tinSeries <= inSeries+1;", file=of)
print("\t\t\tend", file=of)
# print("\t\t\telse begin", file=of)
# print("\t\t\t\tLLinReady <= 0;", file=of)
# print("\t\t\tend", file=of)
print("\t\tend", file=of)
if HRR==2:
    print("\t\telse begin", file=of)
    print("\t\t\tinSeries <= 0;", file=of)
    print("\t\t\treadyForNewDataSeries <= 1;", file=of)
    print("\t\tend", file=of)
else:
    print("\t\telse if(inSeries=={}) begin".format(HRR-1), file=of)
    print("\t\t\tinSeries <= 0;", file=of)
    print("\t\t\treadyForNewDataSeries <= 1;", file=of)
    print("\t\tend", file=of)
    print("\t\telse begin", file=of)
    print("\t\t\tinSeries <= inSeries+1;", file=of)
    print("\t\tend", file=of)
print("\t\tcase(inSeries)", file=of)
print("\t\t\t0: if(inReady) begin".format(i), file=of) #currently more power efficient, can use "begin" for simpler hardware
print("\t\t\t\t", end='', file=of)
for j in range(NMult):
    print("SA{} <= A{}; ".format(j,j), end='', file=of)
print("\n\t\t\t\t", end='', file=of)
for j in range(NMult):
    print("SB{} <= B{}; ".format(j,j), end='', file=of)
if(N>NMult):
    print("\n\t\t\t\t", end='', file=of)
    for j in range(NMult,N):
        print("A{}l <= A{}; ".format(j, j), end='', file=of)
    print("\n\t\t\t\t", end='', file=of)
    for j in range(NMult,N):
        print("B{}l <= B{}; ".format(j, j), end='', file=of)
print("\n\t\t\tend", file=of)
for i in range(1,HRR):
    print("\t\t\t{}: begin".format(i), file=of)
    print("\t\t\t\t", end='', file=of)
    for j in range(NMult):
        k=NMult*i+j
        if k >= N:
            print("SA{} <= 0; ".format(j), end='', file=of)            
        else:
            print("SA{} <= A{}l; ".format(j,k), end='', file=of)
    print("\n\t\t\t\t", end='', file=of)
    for j in range(NMult):
        k=NMult*i+j
        if k >= N:
            print("SB{} <= 0; ".format(j), end='', file=of)            
        else:
            print("SB{} <= B{}l; ".format(j,k), end='', file=of)
    print("\n\t\t\tend", file=of)
print("\t\tendcase", file=of)
print("\tend", file=of)
print("end\n", file=of)

In [ ]:
if lghrr==1:
    print("reg outSeries = 0;", file=of)
else:
    print("reg [{}:0] outSeries = 0;".format(lghrr-1), file=of)
if al==0:
    print("reg signed [(2*IN_WIDTH):0] DPps = 0;", file=of)
else:
    print("reg signed [(2*IN_WIDTH)+{}:0] DPps = 0;".format(al), file=of)
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\toutSeries <= 0;", file=of)
print("\t\tDPps <= 0;", file=of)
print("\t\toutReady <= 0;", file=of)
print("\tend", file=of)

print("\telse if(enable) begin", file=of)
print("\t\tif(outSeries=={}) begin".format(HRR-1), file=of)
if NumOfComponents==1:
    print("\t\t\tif(DPOutReady) begin", file=of)
else:
    print("\t\t\tif(ATOutReady) begin", file=of)
print("\t\t\t\toutReady <= 1;", file=of)
print("\t\t\t\tDP <= DPps+DPpart;", file=of)
print("\t\t\t\toutSeries <= 0;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\t\telse begin", file=of)
print("\t\t\toutReady <= 0;", file=of)
if NumOfComponents==1:
    print("\t\t\tif(DPOutReady) begin", file=of)
else:
    print("\t\t\tif(ATOutReady) begin", file=of)
if HRR==2:
    print("\t\t\t\tDPps <= DPpart;", file=of)
    print("\t\t\t\toutSeries <= 1;", file=of)
else:
    print("\t\t\t\tif(outSeries==0) begin", file=of)
    print("\t\t\t\t\tDPps <= DPpart;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\t\telse begin", file=of)
    print("\t\t\t\t\tDPps <= DPps+DPpart;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\t\toutSeries <= outSeries+1;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\tend", file=of)

# print("\telse if(enable) case(outSeries)", file=of)
# print("\t\t{}: begin".format(HRR-1), file=of)
# if NumOfComponents==1:
#     print("\t\t\tif(DPOutReady) begin", file=of)
# else:
#     print("\t\t\tif(ATOutReady) begin", file=of)
# print("\t\t\t\toutReady <= 1;", file=of)
# print("\t\t\t\tDP <= DPps+DPpart;", file=of)
# print("\t\t\t\toutSeries <= 0;", file=of)
# print("\t\t\tend", file=of)
# print("\t\tend", file=of)
# print("\t\t0: begin", file=of)
# print("\t\t\toutReady <= 0;", file=of)
# if NumOfComponents==1:
#     print("\t\t\tif(DPOutReady) begin", file=of)
# else:
#     print("\t\t\tif(ATOutReady) begin", file=of)
# print("\t\t\t\tDPps <= DPpart;", file=of)
# if HRR==2:
#     print("\t\t\t\toutSeries <= 1;", file=of)
# else:
#     print("\t\t\t\toutSeries <= outSeries+1;", file=of)
# print("\t\t\tend", file=of)
# print("\t\tend", file=of)
# if HRR>2:
#     print("\t\tdefault: begin", file=of)
#     if NumOfComponents==1:
#         print("\t\t\tif(DPOutReady) begin", file=of)
#     else:
#         print("\t\t\tif(ATOutReady) begin", file=of)
#     print("\t\t\t\tDPps <= DPps+DPpart;", file=of)
#     print("\t\t\t\toutSeries <= outSeries+1;", file=of)
#     print("\t\t\tend", file=of)
#     print("\t\tend", file=of)
# print("\tendcase", file=of)

print("end\n", file=of)

In [ ]:
if NumOfComponents==1:
    #combinational earlyOutReady:
    print("assign earlyOutReady = (outSeries=={}) & DPOutReady;\n".format(HRR-1), file=of)
else:
    if _Use_Registered_earlyOutReady_If_Available<=0:
        #combinational earlyOutReady:
        print("assign earlyOutReady = (outSeries=={}) & ATOutReady;\n".format(HRR-1), file=of)
    else:
        #registered earlyOutReady:
        print("reg earlyOutReady;", file=of)
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\tearlyOutReady <= 0;", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif((outSeries=={}) & ATEarlyOutReady) begin".format(HRR-1), file=of)
        print("\t\t\tearlyOutReady <= 1;", file=of)
        print("\t\tend", file=of)
        print("\t\telse begin", file=of)
        print("\t\t\tearlyOutReady <= 0;", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

In [ ]:
print("endmodule", file=of)

In [ ]:
if __Print_To_File>0:
    of.close()